In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("fake_news.csv")
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
data = data.drop(['id'], axis = 1)

In [ ]:
data = data.fillna('')

In [ ]:
data['content'] = data['author']+' '+ data['title']+' '+data['text']

In [ ]:
data = data.drop(['title','author','text'],axis=1)

In [ ]:
data.head()

In [ ]:
data['content'] = data['content'].apply(lambda x:" ".join(x.lower() for x in x.split()))

In [ ]:
# Remove punctuation
data['content'] = data['content'].str.replace('[^\w\s]','')

In [ ]:
# Remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
from nltk.stem import WordNetLemmatizer
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))
data['content'].head()

In [ ]:
X = data[['content']]
y = data['label']

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# splitting into training and testing data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=45, stratify=y)

In [ ]:
#validate the shape of train and test dataset
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(data['content'])
xtrain_tfidf = tfidf_vect.transform(X_train['content'])
xtest_tfidf = tfidf_vect.transform(X_test['content'])

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics   
pclf = PassiveAggressiveClassifier()
pclf.fit(xtrain_tfidf, y_train)
predictions = pclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test, predictions))

In [ ]:
print(metrics.confusion_matrix(y_test,predictions))

In [ ]:
from sklearn.neural_network import MLPClassifier 
mlpclf = MLPClassifier(hidden_layer_sizes=(256,64,16), activation = 'relu', solver = 'adam') 
mlpclf.fit(xtrain_tfidf, y_train) 
predictions = mlpclf.predict(xtest_tfidf) 
print(metrics.classification_report(y_test, predictions))


In [ ]:
print(metrics.confusion_matrix(y_test,predictions))

In [30]:
import pickle
# Save trained model to file
pickle.dump(mlpclf, open("fakenews1.pkl", "wb"))

NameError: name 'mlpclf' is not defined

In [29]:
# load the model from disk
loaded_model = pickle.load(open("fakenews1.pkl", 'rb'))
result = loaded_model.score(X_test.values, y_test.values)
print(result)

NameError: name 'pickle' is not defined